In [0]:
#-*- coding: utf-8 -*-
import cv2
import numpy as np
import glob
import sys

FNAME = 'digits.npz'

machineLearning

5000개의 손글씨를 읽어 각 숫자의 cell을 잘라서 배열에 저장합니다.
그리고 각 배열에 Label작업을 수행한 결과를 numpy파일로 저장합니다. >>라벨 정리하기

In [0]:
def machineLearning():
    img = cv2.imread('images/digits.png')
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    cells = [np.hsplit(row,100) for row in np.vsplit(gray,50)]
    x = np.array(cells)
    train = x[:,:].reshape(-1,400).astype(np.float32)

    k = np.arange(10)
    train_labels = np.repeat(k,500)[:,np.newaxis]

    np.savez(FNAME,train=train,train_labels = train_labels)

resize20

직접 쓴 손글씨를 20X20으로 resize 한 결과를 Return합니다.

In [0]:

def resize20(pimg):
    img = cv2.imread(pimg)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    grayResize = cv2.resize(gray,(20,20))
    ret, thresh = cv2.threshold(grayResize, 125, 255,cv2.THRESH_BINARY_INV)

    cv2.imshow('num',thresh)
    return thresh.reshape(-1,400).astype(np.float32)

loadTrainData

학습을 통해 저장되었던 numpy파일을 load합니다.

In [0]:
def loadTrainData(fname):
    with np.load(fname) as data:
        train = data['train']
        train_labels = data['train_labels']

    return train, train_labels

checkDigit

test data와 학습 data를 이용하여 kNN 알고리즘을 적용하고 그 결과를 return합니다.
먼저 학습을 수행합니다. 학습을 수행하기 위해서 아래와 같이 입력합니다.

In [0]:
def checkDigit(test, train, train_labels):
    knn = cv2.ml.KNearest_create()
    knn.train(train, cv2.ml.ROW_SAMPLE, train_labels)

    ret, result, neighbours, dist = knn.findNearest(test, k=5)

    return result

In [0]:
if __name__ == '__main__':
    if len(sys.argv) == 1:
        print 'option : train or test'
        exit(1)
    elif sys.argv[1] == 'train':
        machineLearning()                                      //먼저 트레이닝
    elif sys.argv[1] == 'test':
        train, train_labels = loadTrainData(FNAME)             //label된 데이터 준비

        saveNpz = False
        for fname in glob.glob('images/num*.png'):             //그림 자체의 fname 마다
            test = resize20(fname)                             //다른 크기로 resize
            result = checkDigit(test, train, train_labels)     //학습

            print result

            k = cv2.waitKey(0)                                 //일단 대기

            if k > 47 and k<58:
                saveNpz = True
                train = np.append(train, test, axis=0)
                newLabel = np.array(int(chr(k))).reshape(-1,1)
                train_labels = np.append(train_labels, newLabel,axis=0)


        cv2.destroyAllWindows()
        if saveNpz:
            np.savez(FNAME,train=train, train_labels=train_labels)
    else:
        print 'unknow option'